In [1]:
import numpy as np
import scipy as sp
from scipy import optimize
import math
import matplotlib.pyplot as plt

In [42]:
class Layer:
    def __init__(self, neurons, activation, input_l = False, output_l = False):
        self.neurons = neurons
        self.activation = activation
        self.input_l = input_l
        self.output_l = output_l
        self.weights = np.array([])
        self.b = np.array([])
    
    def activation_deri(self, x, h = 10**-11):
        return (self.activation(x+h)-self.activation(x))/h

    def __str__(self):
        return f'<Object Layer(neurons={self.neurons}, activation={self.activation}, weights={self.weights}, bias={self.b})>'
    
    def layer_activation(self, prev_activation):
        pre_activation = np.matmul(self.weights, prev_activation) + self.b
        return (pre_activation, self.activation(pre_activation))


In [106]:
class Sequential:
    def __init__(self):
        self.layers = []

    def add(self, layer):
        self.layers.append(layer)
    
    def fit(self, X_train, y_train, alpha, epochs):
        history = {
            'loss': [],
            'accuracy': []
        }
        for i, layer in enumerate(self.layers):
            if not layer.input_l:
                layer.weights = np.random.random((layer.neurons, self.layers[i-1].neurons ))
                layer.b = np.random.random(layer.neurons)
            
        for epoch in range(1, epochs+1):
            for x, y in zip(X_train, y_train):
                self.partial_fit(x, y, alpha)
            
            if epoch % 100 == 0:
                loss = self.calculate_loss(X_train, y_train)
                accuracy = self.calculate_accuracy(X_train, y_train)
                print("EPOCH : ", epoch, " LOSS : ", loss, " ACCURACY : ", accuracy)
                history['loss'].append(loss)
                history['accuracy'].append(accuracy)
        
        return history
    
    def partial_fit(self, x, y, alpha):
        a_l = [x]
        z_l = [x]
        for i, layer in enumerate(self.layers):
            if not layer.input_l:
                z, a = layer.layer_activation(a_l[-1])
                z_l.append(z)
                a_l.append(a)
        
        d = [self.layers[-1].activation_deri(z_l[-1])*(a_l[-1]-y)]
        
        for i in range(len(self.layers)-1, -1, -1):
            if not self.layers[i].output_l and not self.layers[i].input_l:
                d_temp = self.layers[i].activation_deri(z_l[i])* self.layers[i+1].weights.T.dot(d[-1])
                d.append(d_temp)
        
        d.reverse()
        for i, layer in enumerate(self.layers):
            if not layer.input_l:
                a_l[i-1] = a_l[i-1].reshape(1, -1)
                d[i-1] = d[i-1].reshape(1, -1)
                grad = d[i-1].T.dot(np.column_stack((a_l[i-1], [1])))
                
                layer.weights, layer.b = layer.weights - alpha*grad[:, :-1], layer.b - alpha*grad[:, -1]
    
    def calculate_loss(self, X, Y):
        y_pred = self.predict(X)
        loss = (y_pred-Y)**2
        loss = sum(loss)
        return 0.5*loss
    
    def calculate_accuracy(self, X, y):
        count = 0
        y_pred = self.predict(X)
        for i, j in enumerate(y):
            if j[0] == round(y_pred[i][0]):
                count += 1

        return count/len(y)

    def predict(self, input_x):
        outputs = []
        for xi in input_x:
            output = xi
            for layer in self.layers:
                if not layer.input_l:
                    z, output = layer.layer_activation(output)
        
            outputs.append(output)
            
        outputs = np.array(outputs)
        return outputs


In [107]:
def sigmoid(x):
    return 1/(1+np.exp(x))

# Data Preparation

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [27]:
data = pd.read_csv("sample_datasets/breast-cancer-wisconsin.data", header=None, na_values='?')

In [9]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2


In [10]:
data.dropna(axis=0, inplace=True)

In [11]:
x_full = data.drop([0, 10], axis=1)
x_full.columns = [f'feature {i}' for i in x_full]
x_full.head()

,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9
0,5,1,1,1,2,1.0,3,1,1
1,5,4,4,5,7,10.0,3,2,1
2,3,1,1,1,2,2.0,3,1,1
3,6,8,8,1,3,4.0,3,7,1
4,4,1,1,3,2,1.0,3,1,1


In [12]:
y_full = data[10]
y_full.name = 'Target'
y_full = (y_full -2)/2
y_full

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
694    0.0
695    0.0
696    1.0
697    1.0
698    1.0
Name: Target, Length: 683, dtype: float64

In [13]:
x_train, x_valid, y_train, y_valid = train_test_split(x_full, y_full)

In [14]:
print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(512, 9)
(171, 9)
(512,)
(171,)


In [15]:
x_train.head()

,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9
331,5,1,1,1,2,1.0,3,1,2
237,9,8,8,5,6,2.0,4,10,4
152,10,10,8,6,4,5.0,8,10,1
37,6,2,1,1,1,1.0,7,1,1
68,8,3,8,3,4,9.0,8,9,8


In [16]:
y_train.head()

331    0.0
237    1.0
152    1.0
37     0.0
68     1.0
Name: Target, dtype: float64

In [17]:
scalar = StandardScaler()
x_train = scalar.fit_transform(x_train)
x_valid = scalar.transform(x_valid)

In [18]:
pd.DataFrame(x_train)

,0,1,2,3,4,5,6,7,8
0,0.152743,-0.715036,-0.753772,-0.660590,-0.575816,-0.724845,-0.195832,-0.635605,0.256978
1,1.543042,1.575124,1.591296,0.725566,1.258198,-0.452762,0.221943,2.302077,1.453097
2,1.890616,2.229456,1.591296,1.072104,0.341191,0.363485,1.893041,2.302077,-0.341081
3,0.500318,-0.387870,-0.753772,-0.660590,-1.034319,-0.724845,1.475266,-0.635605,-0.341081
4,1.195467,-0.060705,1.591296,0.032488,0.341191,1.451815,1.893041,1.975668,3.845333
...,...,...,...,...,...,...,...,...,...
507,1.890616,0.920793,0.251257,-0.660590,-0.117312,0.091403,-0.195832,-0.309196,0.855038
508,0.152743,0.266461,0.251257,0.725566,1.716701,1.723898,-0.195832,-0.309196,-0.341081
509,1.890616,0.266461,-0.083752,2.458259,0.341191,1.723898,2.728590,-0.635605,-0.341081
510,-0.194832,-0.715036,-0.753772,-0.660590,-0.575816,-0.724845,-1.031381,-0.635605,-0.341081


# My NN

In [108]:
model = Sequential()
model.add(Layer(9, sigmoid, input_l=True))

model.add(Layer(10, sigmoid))
model.add(Layer(10, sigmoid))
model.add(Layer(1, sigmoid, output_l=True))

In [109]:
y_train = np.array(y_train)
y_train = y_train.reshape(len(y_train), -1)

In [112]:
r = model.fit(x_train, y_train, epochs=5000, alpha=0.1)

EPOCH :  100  LOSS :  [4.36714601]  ACCURACY :  0.982421875
EPOCH :  200  LOSS :  [3.83436902]  ACCURACY :  0.984375
EPOCH :  300  LOSS :  [3.13369745]  ACCURACY :  0.990234375
EPOCH :  400  LOSS :  [2.50138604]  ACCURACY :  0.9921875
EPOCH :  500  LOSS :  [2.25051655]  ACCURACY :  0.9921875
EPOCH :  600  LOSS :  [2.14643709]  ACCURACY :  0.9921875
EPOCH :  700  LOSS :  [2.09687968]  ACCURACY :  0.9921875
EPOCH :  800  LOSS :  [2.06980734]  ACCURACY :  0.9921875
EPOCH :  900  LOSS :  [2.05336844]  ACCURACY :  0.9921875
EPOCH :  1000  LOSS :  [2.04257333]  ACCURACY :  0.9921875
EPOCH :  1100  LOSS :  [2.0350544]  ACCURACY :  0.9921875
EPOCH :  1200  LOSS :  [2.02957324]  ACCURACY :  0.9921875
EPOCH :  1300  LOSS :  [2.02543221]  ACCURACY :  0.9921875
EPOCH :  1400  LOSS :  [2.02221169]  ACCURACY :  0.9921875
EPOCH :  1500  LOSS :  [2.01964685]  ACCURACY :  0.9921875
EPOCH :  1600  LOSS :  [2.0175636]  ACCURACY :  0.9921875
EPOCH :  1700  LOSS :  [2.01584236]  ACCURACY :  0.9921875
EPOCH

In [113]:
y_valid = np.array(y_valid)
y_valid = y_valid.reshape((len(y_valid), -1))

y_pred = model.predict(x_valid)

count = 0
for i, j in enumerate(y_valid):
    if j[0] == round(y_pred[i][0]):
        count += 1

print('Accuracy : ', count/len(y_valid))

Accuracy :  0.9415204678362573
